This package implements the basic machinery for decorrelated backpropagation.

In [ ]:
import torch
from decorrelation import lower_triangular_correlation, DecorrelationPatch2d, DecorrelationFC
import matplotlib.pyplot as plt

: 

We start by showing effective decorrelation of correlated input

In [ ]:
# generate [N, M] correlated input
d = torch.distributions.MultivariateNormal(torch.zeros(800), 0.1 * torch.eye(800) + 0.9 * torch.ones((800, 800)))
x = d.sample((100,))
plt.imshow((x @ x.T) / len(x))


: 

Decorrelation of correlated input represented in convolutional [N, C, H, W] form

In [ ]:
# transform to [N, C, H, W] correlated input
x = x.reshape(100,2,20,20)


model = DecorrelationPatch2d(2, kernel_size=(3,3))

plt.figure()
for i in range(100):
    y = model.forward(x)
    model.update()
    model.R -= 1e-2 * model.R.grad
    # print(mean_correlation([model]).numpy()) # mean correlation over batches and patches
    plt.subplot(10,10,i+1)
    plt.imshow(model.correlation(model.output))
    print(lower_triangular_correlation(DecorrelationFC.correlation(model.flatten(model.output))).numpy()) # mean correlation of the whole feature map
plt.savefig("debug.png")

: 